In [84]:
import numpy as np
import pandas as pd

In [213]:
inputs_schedule_file_path = r"tp_schedule_am_bus.csv"

inputs_tline_formats = r"inputs_tline_formats.csv"

In [214]:
output_filepath = r"BUS_lines_2.txt"
# ..\temp_data\outputs\

In [215]:
tp_schedule_am_bus = pd.read_csv(inputs_schedule_file_path)

In [216]:
line_formats = pd.read_csv(inputs_tline_formats).set_index('line_key')['format_string']


In [217]:
def export_to_transaction_files_v2(model_schedule, lines_file, line_formats):
    """
    consecutive stops associated to a single node should be skipped from the line file. However it is recommended to
    keep the information of the stop that is skipped in the output line-file as a comment (add a c in front of the line). 
    When a stop is commented out (skipped) the `time from the last stop` (us3 value in the output) should include the time requires to go through 
    the stops that are skipped.
    TODO: Correct the `time from the last stop` in dummy nodes. Dummy nodes are thoes with the marker value of '#'. currently the value of 
    `time from the last stop` for dummy nodes are equal to zero. This value should be set equal to the value of the next row [that is not dummy node].
    
    
    """

    model_schedule.sort_values(['time_period_code',
                                'line_id',
                                'route_id',
                                'direction',
                                'stop_order'],
                               ascending=True, inplace=True)
    model_schedule["shifted_node_id"] = \
        model_schedule.groupby(['time_period_code', 'line_id', 'route_id', 'direction'])[
            'node_id'].shift(1)
    model_schedule["is_prev_node"] = np.where(model_schedule["node_id"] == model_schedule["shifted_node_id"],
                                              True, False)
    model_schedule['subgroup'] = (model_schedule['is_prev_node'] != model_schedule['is_prev_node'].shift(1)).cumsum()
    model_schedule['subgroup_time_from_last_stop'] = model_schedule['time_from_last_stop_minutes'].groupby(
        model_schedule['subgroup']).transform('sum')
    model_schedule["subgroup_time_from_last_stop"] = np.where(model_schedule["is_prev_node"],
                                                              model_schedule["subgroup_time_from_last_stop"], np.NAN)
    model_schedule["time_from_last_stop_minutes"] = np.where(
        ((model_schedule["is_prev_node"] == False) & (model_schedule["is_prev_node"].shift(1) == True)),
        model_schedule["subgroup_time_from_last_stop"].shift(1) + model_schedule['time_from_last_stop_minutes'],
        model_schedule['time_from_last_stop_minutes']) 
    
    
     ## TO-DO: Done from here
        
    model_schedule["hash_indic"] = np.where(model_schedule["marker"] == "#", True, False)
    model_schedule["hash_counter"] = np.where(model_schedule["marker"] == "#", 1, 0)
        
    ## Finding the number of dummies in each group of dummies
    model_schedule['hash_group'] = (model_schedule['hash_indic'] != model_schedule['hash_indic'].shift(1)).cumsum()
    model_schedule['hash_group_count'] = model_schedule.groupby(by=['hash_group'])['line_id'].transform('count')

    model_schedule["hash_counter"] = np.where(model_schedule["hash_indic"] == True, model_schedule['hash_group_count'], 0)   

    
    ## Allocate correct hash_counter to all hashes of each group plus the first positive node after each group of dummies   
    model_schedule["modif_hash_indic"] = np.where(np.logical_or(model_schedule["hash_indic"],
                                                       model_schedule["hash_indic"].shift(1, fill_value=0)), True, False)
    positive_indic = pd.Series(np.logical_xor(model_schedule["modif_hash_indic"], model_schedule["hash_indic"]))
    
    model_schedule["hash_counter"] = np.where(positive_indic,
                                              model_schedule["hash_counter"].shift(1),
                                              0) + model_schedule["hash_counter"]
    
    model_schedule["hash_group"] = np.where(positive_indic,
                                          model_schedule["hash_group"].shift(1),
                                          np.where(model_schedule["hash_indic"] == True,
                                                  model_schedule["hash_group"], 0))
    
    # Dommy groups' time modification
    model_schedule["time_from_last_stop_minutes"] = np.where(model_schedule["hash_group"] != 0, 
                                                             model_schedule["time_from_last_stop_minutes"].groupby(model_schedule["hash_group"]).transform(lambda x: x.iloc[-1]),
                                                             model_schedule["time_from_last_stop_minutes"])
    
    model_schedule["time_from_last_stop_minutes"] = model_schedule["time_from_last_stop_minutes"]/(model_schedule["hash_counter"]+1)

    
    with open(lines_file, 'w') as f:
        f.write("t lines\n")
        for i, line in model_schedule.iterrows():
            if line['stop_order'] == 1:
                f.write("c ---------------------------------------\n")
                add_transit_string = line_formats.loc['add_transit_line'].format(l=line).replace("'n'", "n")
                f.write(add_transit_string)
                f.write('\n')
                f.write(line_formats.loc['transit_line_details'].format(l=line))
                f.write('\n')
                f.write(line_formats.loc['first_node'].format(l=line))
                f.write('\n')
            else:
                string = line_formats.loc['normal_stop_line'].format(l=line).replace("'", " ")
                if line["is_prev_node"]:
                    string = 'c ' + string
                f.write(string)
                f.write('\n')


In [218]:
mode_code ='b'
mode_line_file = 'BUS'

In [219]:
tp_schedule = tp_schedule_am_bus.copy()

In [220]:
tp_schedule = tp_schedule.astype({'allow_boardings': 'bool', 'allow_alightings': 'bool', 'stop_id':'str'})

In [221]:
#apply some formatting to certain columns in schedule file.
tp_schedule["stop_id"] = tp_schedule['stop_id'].apply(lambda x: x.rjust(6,' ')) #-------
tp_schedule["stop_name"] =tp_schedule['stop_name'].apply(lambda x: x.replace("'", ""))#-------
tp_schedule = tp_schedule.sort_values(by=['line_id', 'stop_order'])

In [222]:
print("Writing {} for mode {} in time period {}".format(output_filepath, mode_code, '2')) 

Writing BUS_lines_2.txt for mode b in time period 2


In [223]:
export_to_transaction_files_v2(tp_schedule, output_filepath, line_formats)